In [1]:
# Librairies utiles
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from IPython.core.display import Image, display
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.grid_search import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

import warnings
# suppression des messages d'erreurs
warnings.filterwarnings('ignore')


/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()
import numpy as np


def draw(ylim, clf, i_max=9, j_max=9, ecart_i=50, ecart_j=50):
    # Configuration de la présentation des axes
    plt.ylim(ylim)

    axes = plt.gca()

    # Récupération de tout les scores
    scores = [ mean_score.mean() for params, mean_score, mean_score in clf.grid_scores_ ]

    # Scores avec traitement supplémentaires
    scores2 = []
    legend2 = []
    colors2 = ['#e6194b', '#3cb44b','#ffe119','#0082c8','#f58231','#911eb4','#46f0f0','#f032e6','#d2f53c','#000000']
    x = np.arange(0,i_max*ecart_i,ecart_i)
    for i in range (0,i_max-1) :
        scores2.append([])
        for j in range (0,j_max) :
            scores2[i].append(scores[i * i_max + j])

        plt.plot(x, scores2[i], color=colors2[i])
        legend2.append("Hidden Layer Size : (" + str((i+1)*ecart_i) +", x)")


    plt.legend(legend2) 

    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)

    plt.show()




# Dataset BRUT

In [3]:
# récupération de variables qui ont été stockées dans le noteBook "Pré-traitement" (DATASET BRUT)
%store -r activity_dataset_full
%store -r colonnes_source
%store -r colonnes_cible
%store -r source
%store -r cible

In [4]:
# cration d'un jeu de données de test et d'entrainement
source_train, source_test, cible_train, cible_test = train_test_split(source, cible, test_size=0.7)


In [5]:
#[print('classe ',i,' : ',sum(cible_train==i)) for i in cible_train.values]

display(pd.Series.value_counts(activity_dataset_full['Action']))
display(source_train, source_test, cible_train, cible_test)


S'assoir (depuis position allongée)    66
Courrir                                60
S'allonger (depuis position assise)    59
Ne rien faire                          57
Marcher                                57
S'allonger (depuis position debout)    56
Sauter                                 56
S'assoir (chaise)                      52
Se lever (chaise)                      51
Se lever (depuis position allongée)    51
Name: Action, dtype: int64

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,z14,z15,z16,z17,z18,z19,z20,Lieux_Ecole,Lieux_Exterieur,Lieux_Maison
322,2.217758,1.938828,1.614410,-0.259048,-2.186371,-4.257355,5.806671,2.388947,-0.278198,-5.599319,...,10.956436,2.158936,8.423370,4.484909,11.930878,5.605392,8.011566,0,0,1
378,-9.949570,-9.461166,-8.595657,-10.406876,-12.001404,-11.078445,-9.601227,-10.548126,-9.718536,-6.736557,...,11.140778,10.657150,7.471664,6.517578,5.178024,4.636932,4.629745,0,0,1
385,-1.359177,-1.362778,0.964386,-1.003647,0.526260,0.277252,-1.028778,-1.332840,-0.632538,0.313171,...,3.219574,5.085846,13.029816,12.179871,7.792496,4.896713,3.350052,0,0,1
408,-0.573883,-0.403900,0.355072,-1.949356,0.188675,1.680252,2.908478,3.547729,5.002213,3.904465,...,-6.468567,-0.473495,1.954239,0.092743,0.717621,0.089142,2.216400,0,0,1
161,-1.893082,-2.443756,-1.736267,-1.130539,0.237747,-0.753448,-0.497269,2.296768,-0.840836,-0.673248,...,12.962769,19.339722,9.701874,4.992477,6.084229,8.744186,6.810867,1,0,0
438,9.498520,8.509720,9.359665,8.240372,9.397964,11.428253,11.355225,13.681198,8.745544,5.485840,...,5.650879,7.369919,3.081894,3.128586,3.025635,3.293793,3.157318,0,0,1
507,0.110855,-0.105820,-0.068710,0.039032,0.022278,0.013901,0.175507,0.316757,-0.405090,0.005508,...,8.337173,7.967270,7.478851,6.372726,5.308502,4.500473,3.511658,0,0,1
522,-0.942596,-1.223907,-1.287354,-1.272995,-1.142502,-1.197571,-1.262222,-1.286163,-1.274185,-1.306503,...,8.569412,8.485611,8.398224,8.620880,8.732224,8.665176,8.774109,0,0,1
203,0.120438,1.173889,-0.065109,-0.162079,0.007904,0.045013,-1.008423,-0.995270,0.249725,1.465973,...,5.255829,5.336044,3.573914,3.527222,2.870010,3.051971,2.723969,0,0,1
368,0.356262,0.636383,-0.935410,0.292816,1.193039,-0.869568,0.124023,2.579285,-0.505646,-0.846817,...,2.861633,6.311676,4.603409,9.402588,6.590607,3.581085,4.739883,0,0,1


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,z14,z15,z16,z17,z18,z19,z20,Lieux_Ecole,Lieux_Exterieur,Lieux_Maison
17,0.394577,0.344299,0.212616,0.132401,-0.158493,-0.231506,-0.192001,-0.024414,0.141983,1.056564,...,5.107391,6.077042,6.263794,6.832413,6.585815,6.134506,6.831223,0,0,1
335,0.787216,0.888977,0.792007,0.855453,0.744125,0.884186,0.772858,0.844681,0.768066,0.916504,...,8.669968,8.583771,8.592163,8.648422,8.611313,8.646027,8.663986,0,0,1
23,-0.305725,-0.620575,-0.432632,-0.675644,-1.118561,-1.645294,-2.054703,-2.510788,-2.874710,-2.416214,...,3.624191,-1.078018,0.239990,-2.702484,-2.357727,-2.526520,-2.597153,0,0,1
58,-0.455368,-0.651688,-1.283768,-0.925827,-0.367981,-0.152496,1.274445,3.947571,5.712097,6.004181,...,-3.243576,-2.148239,-0.587219,-0.674606,-0.109573,0.629044,0.344131,0,0,1
209,-0.145325,0.489136,-0.054337,0.021072,-0.180038,-0.682816,-0.359604,-0.095047,0.110855,-0.336853,...,2.653336,3.013672,2.964584,2.391174,4.475327,1.000137,2.374420,0,0,1
25,-0.208771,-0.918655,-1.654861,-1.908646,-1.877533,-2.314468,-1.933792,-1.547119,1.896927,3.400497,...,2.740723,-1.033737,-2.172180,1.553207,2.441452,0.655380,-0.518982,0,0,1
42,9.632599,10.198822,10.613022,11.786179,11.515640,12.029190,9.125031,7.931519,6.087982,3.519012,...,4.269424,5.642502,5.786148,5.011627,5.546738,5.871140,5.784958,0,0,1
434,9.564362,9.386002,9.389587,9.569153,9.844482,12.005249,13.468109,13.071869,4.829834,10.517258,...,11.253311,12.469559,9.844330,8.533493,5.683197,3.548767,2.430679,0,0,1
426,-9.792755,-9.755646,-9.458771,-9.169067,-10.374542,-12.944717,-12.329407,-12.415604,-9.549744,-8.064148,...,-0.156250,2.805374,2.690445,3.456589,4.615387,4.933823,4.632141,0,0,1
62,-0.644516,-0.654083,0.598083,2.038193,3.449570,4.212128,3.985870,5.810257,9.669708,11.674850,...,3.001694,1.631012,1.954239,2.556381,1.973389,1.926697,1.410751,0,0,1


,Action
322,Courrir
378,Se lever (depuis position allongée)
385,Se lever (depuis position allongée)
408,S'allonger (depuis position assise)
161,Marcher
438,S'assoir (depuis position allongée)
507,S'assoir (chaise)
522,Ne rien faire
203,S'assoir (chaise)
368,Se lever (depuis position allongée)


,Action
17,Se lever (chaise)
335,Ne rien faire
23,S'allonger (depuis position debout)
58,S'allonger (depuis position assise)
209,Se lever (chaise)
25,S'allonger (depuis position debout)
42,S'assoir (depuis position allongée)
434,S'assoir (depuis position allongée)
426,S'assoir (depuis position allongée)
62,S'allonger (depuis position assise)


In [6]:
# Configuration des paramètres que l'on souhaite tester
tuned_parameters = [{
   'n_neighbors': np.array(range (1,100))
}]
scores = ['accuracy']

warnings.filterwarnings('ignore')

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(KNeighborsClassifier(), tuned_parameters, cv=5, scoring=score,verbose=2)
    clf.fit(source_train, cible_train.values.ravel())
    
    
    print("Best parameters set found on development set:")
    print()
    print(clf.best_estimator_)
    
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf.grid_scores_:
          
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() / 2, params))
    print()
    
    print("Detailed classification report:")
    print()
    cible_true, cible_pred = cible_test, clf.predict(source_test)
    print(classification_report(cible_true, cible_pred))
    print()
    
    

# Tuning hyper-parameters for accuracy

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=2 ...................................................
[CV] .......................................... n_neighbors=2 -   0.0s
[CV] n_neighbors=2 ...........................

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] n_neighbors=14 ..................................................
[CV] ......................................... n_neighbors=14 -   0.0s
[CV] n_neighbors=14 ..................................................
[CV] ......................................... n_neighbors=14 -   0.0s
[CV] n_neighbors=14 ..................................................
[CV] ......................................... n_neighbors=14 -   0.0s
[CV] n_neighbors=14 ..................................................
[CV] ......................................... n_neighbors=14 -   0.0s
[CV] n_neighbors=15 ..................................................
[CV] ......................................... n_neighbors=15 -   0.0s
[CV] n_neighbors=15 ..................................................
[CV] ......................................... n_neighbors=15 -   0.0s
[CV] n_neighbors=15 ..................................................
[CV] ......................................... n_neighbors=15 -   0.0s
[CV] n

[CV] ......................................... n_neighbors=27 -   0.0s
[CV] n_neighbors=28 ..................................................
[CV] ......................................... n_neighbors=28 -   0.0s
[CV] n_neighbors=28 ..................................................
[CV] ......................................... n_neighbors=28 -   0.0s
[CV] n_neighbors=28 ..................................................
[CV] ......................................... n_neighbors=28 -   0.0s
[CV] n_neighbors=28 ..................................................
[CV] ......................................... n_neighbors=28 -   0.0s
[CV] n_neighbors=28 ..................................................
[CV] ......................................... n_neighbors=28 -   0.0s
[CV] n_neighbors=29 ..................................................
[CV] ......................................... n_neighbors=29 -   0.0s
[CV] n_neighbors=29 ..................................................
[CV] .

[CV] ......................................... n_neighbors=51 -   0.0s
[CV] n_neighbors=51 ..................................................
[CV] ......................................... n_neighbors=51 -   0.0s
[CV] n_neighbors=51 ..................................................
[CV] ......................................... n_neighbors=51 -   0.0s
[CV] n_neighbors=51 ..................................................
[CV] ......................................... n_neighbors=51 -   0.0s
[CV] n_neighbors=52 ..................................................
[CV] ......................................... n_neighbors=52 -   0.0s
[CV] n_neighbors=52 ..................................................
[CV] ......................................... n_neighbors=52 -   0.0s
[CV] n_neighbors=52 ..................................................
[CV] ......................................... n_neighbors=52 -   0.0s
[CV] n_neighbors=52 ..................................................
[CV] .

[CV] ......................................... n_neighbors=63 -   0.0s
[CV] n_neighbors=64 ..................................................
[CV] ......................................... n_neighbors=64 -   0.0s
[CV] n_neighbors=64 ..................................................
[CV] ......................................... n_neighbors=64 -   0.0s
[CV] n_neighbors=64 ..................................................
[CV] ......................................... n_neighbors=64 -   0.0s
[CV] n_neighbors=64 ..................................................
[CV] ......................................... n_neighbors=64 -   0.0s
[CV] n_neighbors=64 ..................................................
[CV] ......................................... n_neighbors=64 -   0.0s
[CV] n_neighbors=65 ..................................................
[CV] ......................................... n_neighbors=65 -   0.0s
[CV] n_neighbors=65 ..................................................
[CV] .

[CV] ......................................... n_neighbors=86 -   0.0s
[CV] n_neighbors=86 ..................................................
[CV] ......................................... n_neighbors=86 -   0.0s
[CV] n_neighbors=86 ..................................................
[CV] ......................................... n_neighbors=86 -   0.0s
[CV] n_neighbors=86 ..................................................
[CV] ......................................... n_neighbors=86 -   0.0s
[CV] n_neighbors=87 ..................................................
[CV] ......................................... n_neighbors=87 -   0.0s
[CV] n_neighbors=87 ..................................................
[CV] ......................................... n_neighbors=87 -   0.0s
[CV] n_neighbors=87 ..................................................
[CV] ......................................... n_neighbors=87 -   0.0s
[CV] n_neighbors=87 ..................................................
[CV] .

[CV] ......................................... n_neighbors=99 -   0.0s
Best parameters set found on development set:

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

Grid scores on development set:

0.692 (+/-0.025) for {'n_neighbors': 1}
0.592 (+/-0.023) for {'n_neighbors': 2}
0.633 (+/-0.017) for {'n_neighbors': 3}
0.562 (+/-0.015) for {'n_neighbors': 4}
0.544 (+/-0.012) for {'n_neighbors': 5}
0.521 (+/-0.018) for {'n_neighbors': 6}
0.491 (+/-0.017) for {'n_neighbors': 7}
0.479 (+/-0.023) for {'n_neighbors': 8}
0.432 (+/-0.019) for {'n_neighbors': 9}
0.426 (+/-0.009) for {'n_neighbors': 10}
0.402 (+/-0.015) for {'n_neighbors': 11}
0.402 (+/-0.003) for {'n_neighbors': 12}
0.426 (+/-0.017) for {'n_neighbors': 13}
0.379 (+/-0.017) for {'n_neighbors': 14}
0.402 (+/-0.012) for {'n_neighbors': 15}
0.385 (+/-0.015) for {'n_neighbors': 16}
0.361 (+/-0.013) for {'n_neighbors': 

[Parallel(n_jobs=1)]: Done 495 out of 495 | elapsed:    1.6s finished


In [7]:
# Affichage des courbes
# draw([0.6,0.8], clf, i_max, j_max, ecart_i, ecart_j)

In [8]:
# warnings.filterwarnings('ignore')

#clf_demo = 

# clf_demo.fit(source_train, cible_train.values.ravel())


In [9]:
#warnings.filterwarnings('ignore')

#for i in range(0,20):
#    print("prédit : ", clf_demo.predict(source_test.values[i]),"\n attendu : ",cible_test.values[i])


# Dataset ANALYSE

In [10]:
# récupération des variables qui ont été stockées dans le noteBook "Pré-traitement" (DATASET ANALYSE)
%store -r activity_dataset_analysed
%store -r colonnes_source_analysed
%store -r colonnes_cible_analysed
%store -r source_analysed
%store -r cible_analysed


In [11]:
# cration d'un jeu de données de test et d'entrainement
source_train_analysed, source_test_analysed, cible_train_analysed, cible_test_analysed = train_test_split(source_analysed, cible_analysed, test_size=0.7)


In [12]:
#[print('classe ',i,' : ',sum(cible_train==i)) for i in cible_train.values]

display(pd.Series.value_counts(activity_dataset_analysed['Action']))
display(source_train_analysed, source_test_analysed, cible_train_analysed, cible_test_analysed)


S'assoir (depuis position allongée)    66
Courrir                                60
S'allonger (depuis position assise)    59
Ne rien faire                          57
Marcher                                57
S'allonger (depuis position debout)    56
Sauter                                 56
S'assoir (chaise)                      52
Se lever (chaise)                      51
Se lever (depuis position allongée)    51
Name: Action, dtype: int64

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,max_z,max_xyz,min_x,min_y,min_z,min_xyz,ecart_type_x,ecart_type_y,ecart_type_z,ecart_type_xyz
387,-0.668457,-0.435013,1.515060,-0.597824,-3.886261,-2.940552,-1.154480,-0.603806,-0.937805,-0.710358,...,11.836304,11.836304,-3.886261,3.233154,-13.003540,-13.003540,1.249402,2.717879,7.575721,5.803572
330,0.653152,1.282822,1.118820,1.703003,0.820740,1.029037,0.806381,0.808762,0.893768,1.157120,...,4.361603,9.608917,-0.266235,8.860718,2.834106,-0.266235,0.458586,0.193413,0.371885,3.528077
42,9.632599,10.198822,10.613022,11.786179,11.515640,12.029190,9.125031,7.931519,6.087982,3.519012,...,5.871140,12.029190,-1.757813,0.223633,0.226822,-1.757813,5.219849,3.515000,1.936231,3.792053
317,0.829117,0.006714,2.004669,1.395355,0.364639,-0.701965,0.956009,2.445206,0.967987,2.241699,...,16.786316,16.786316,-4.041885,2.631012,-3.832550,-4.041885,2.364577,2.357878,4.690260,4.469085
20,-1.976883,-2.222290,-1.747040,1.154724,-1.324463,-3.664795,-1.148499,7.558029,2.648712,3.550125,...,11.436462,11.436462,-3.664795,-1.476242,1.465820,-3.664795,5.171379,3.140237,2.983531,4.156304
348,0.491531,0.733353,0.316757,0.012695,0.204239,0.568146,0.395767,0.428085,0.191071,0.112061,...,3.806152,9.599335,-0.083069,9.015152,3.068726,-0.083069,0.231009,0.158047,0.252604,3.761887
347,0.460419,0.054596,0.279648,-0.231506,0.380203,-1.012024,0.325134,0.760880,0.896164,0.034241,...,4.355621,9.793259,-1.012024,8.715866,2.940643,-1.012024,0.428309,0.236312,0.382910,3.710444
156,-4.928940,-6.657547,-10.819870,-7.399750,-7.203430,-8.774017,-8.639954,-8.995483,-9.299545,-7.919296,...,8.316818,8.316818,-10.819870,-12.118469,0.153793,-12.118469,2.101397,2.761322,2.324230,5.507750
51,-0.449387,-0.201584,0.358658,1.639557,4.011017,4.584427,4.167831,1.623993,2.036987,5.048904,...,8.498779,16.483612,-0.449387,-3.410751,-5.162537,-5.162537,4.525748,3.966477,4.752761,4.594495
73,-1.053925,-0.616974,-1.460938,-0.933014,-0.512833,-0.691193,-1.263412,-0.887527,-1.118561,-0.072296,...,11.377808,11.377808,-1.821259,2.884796,7.217880,-1.821259,0.578150,0.793254,1.213591,3.987088


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,max_z,max_xyz,min_x,min_y,min_z,min_xyz,ecart_type_x,ecart_type_y,ecart_type_z,ecart_type_xyz
520,0.043823,-0.680420,-0.769012,-0.480515,-3.147644,-1.877533,-1.422623,-1.786545,-0.352417,-1.768585,...,10.776871,10.776871,-4.207077,6.529968,4.362793,-4.207077,1.018248,0.907678,1.308638,4.153375
288,-0.215942,-0.694794,-0.087860,0.090515,0.807571,1.336685,1.634766,0.575333,0.823135,0.821930,...,13.074097,13.074097,-4.711060,0.838943,2.081131,-4.711060,1.908214,2.751356,2.587233,4.269875
56,0.067764,0.420914,-0.286575,-0.254257,0.198242,-0.047165,0.200638,0.237747,0.753708,0.094101,...,5.224716,9.382660,-0.812103,0.682129,-12.185928,-12.185928,0.632186,3.252027,7.009513,5.834202
476,0.130020,0.216202,0.422104,-0.251862,-0.144119,0.708206,0.942841,-0.607407,-2.003220,-2.602966,...,8.152817,14.002274,-15.323364,-2.138245,-6.014862,-15.323364,5.300208,4.490796,3.811287,6.028593
448,0.162338,0.036636,0.041428,-0.069901,-0.020828,-0.132156,-0.020828,-0.256653,0.277252,0.807571,...,14.159866,14.159866,-2.721481,4.153717,2.055984,-2.721481,0.957568,2.094282,2.705071,4.156050
245,-0.122574,0.660324,0.331131,-0.316513,-0.241089,-2.253418,2.441620,3.194595,-1.251434,0.215012,...,38.917114,38.917114,-2.339600,-0.586792,-5.603073,-5.603073,1.402370,3.187362,10.037542,7.100020
463,-1.379532,-1.088638,-1.061096,-1.070679,-1.168839,-1.099411,-1.014420,-0.982101,-1.019211,-1.034760,...,10.240570,10.240570,-1.379532,1.482986,8.957275,-1.379532,0.624402,0.393860,0.276719,4.405321
324,2.241699,2.253662,1.784409,3.129944,0.498718,2.192612,6.372894,7.081573,2.447601,0.016281,...,16.227264,16.867874,-6.586914,-3.822556,-2.411591,-6.586914,5.252613,4.835471,4.857456,5.107017
418,-0.316513,0.034241,-4.665573,-0.747467,0.710602,1.583298,2.270432,3.287964,3.524994,8.266708,...,5.723907,15.169189,-4.665573,-1.780304,-5.867630,-5.867630,5.513705,4.588865,3.173427,4.948919
559,0.090515,-0.144119,-0.965332,-0.241089,-1.191589,1.597656,1.900528,1.076920,1.371399,0.716599,...,3.930649,10.075775,-1.191589,2.779449,-11.115723,-11.115723,0.941126,2.495702,5.007010,5.630115


,Action
387,Se lever (depuis position allongée)
330,Ne rien faire
42,S'assoir (depuis position allongée)
317,Courrir
20,S'allonger (depuis position debout)
348,Ne rien faire
347,Ne rien faire
156,Marcher
51,S'allonger (depuis position assise)
73,Marcher


,Action
520,Ne rien faire
288,S'allonger (depuis position debout)
56,S'allonger (depuis position assise)
476,S'allonger (depuis position debout)
448,S'assoir (chaise)
245,Sauter
463,Ne rien faire
324,Courrir
418,S'allonger (depuis position assise)
559,S'assoir (depuis position allongée)


In [13]:
# Configuration des paramètres que l'on souhaite tester
tuned_parameters = [{
   'n_neighbors': np.array(range (1,100))
}]
scores = ['accuracy']

warnings.filterwarnings('ignore')

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf_analysed = GridSearchCV(KNeighborsClassifier(), tuned_parameters, cv=5, scoring=score,verbose=2)
    clf_analysed.fit(source_train_analysed, cible_train_analysed.values.ravel())
    
    
    print("Best parameters set found on development set:")
    print()
    print(clf_analysed.best_estimator_)
    
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf_analysed.grid_scores_:
          
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() / 2, params))
    print()
    
    print("Detailed classification report:")
    print()
    cible_true_analysed, cible_pred_analysed = cible_test_analysed, clf_analysed.predict(source_test_analysed)
    print(classification_report(cible_true_analysed, cible_pred_analysed))
    print()

# Tuning hyper-parameters for accuracy

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=2 ...................................................
[CV] .......................................... n_neighbors=2 -   0.0s
[CV] n_neighbors=2 ...........................

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ......................................... n_neighbors=13 -   0.0s
[CV] n_neighbors=13 ..................................................
[CV] ......................................... n_neighbors=13 -   0.0s
[CV] n_neighbors=13 ..................................................
[CV] ......................................... n_neighbors=13 -   0.0s
[CV] n_neighbors=13 ..................................................
[CV] ......................................... n_neighbors=13 -   0.0s
[CV] n_neighbors=13 ..................................................
[CV] ......................................... n_neighbors=13 -   0.0s
[CV] n_neighbors=14 ..................................................
[CV] ......................................... n_neighbors=14 -   0.0s
[CV] n_neighbors=14 ..................................................
[CV] ......................................... n_neighbors=14 -   0.0s
[CV] n_neighbors=14 ..................................................
[CV] .

[CV] ......................................... n_neighbors=25 -   0.0s
[CV] n_neighbors=25 ..................................................
[CV] ......................................... n_neighbors=25 -   0.0s
[CV] n_neighbors=25 ..................................................
[CV] ......................................... n_neighbors=25 -   0.0s
[CV] n_neighbors=26 ..................................................
[CV] ......................................... n_neighbors=26 -   0.0s
[CV] n_neighbors=26 ..................................................
[CV] ......................................... n_neighbors=26 -   0.0s
[CV] n_neighbors=26 ..................................................
[CV] ......................................... n_neighbors=26 -   0.0s
[CV] n_neighbors=26 ..................................................
[CV] ......................................... n_neighbors=26 -   0.0s
[CV] n_neighbors=26 ..................................................
[CV] .

[CV] n_neighbors=37 ..................................................
[CV] ......................................... n_neighbors=37 -   0.0s
[CV] n_neighbors=37 ..................................................
[CV] ......................................... n_neighbors=37 -   0.0s
[CV] n_neighbors=37 ..................................................
[CV] ......................................... n_neighbors=37 -   0.0s
[CV] n_neighbors=37 ..................................................
[CV] ......................................... n_neighbors=37 -   0.0s
[CV] n_neighbors=38 ..................................................
[CV] ......................................... n_neighbors=38 -   0.0s
[CV] n_neighbors=38 ..................................................
[CV] ......................................... n_neighbors=38 -   0.0s
[CV] n_neighbors=38 ..................................................
[CV] ......................................... n_neighbors=38 -   0.0s
[CV] n

[CV] ......................................... n_neighbors=61 -   0.0s
[CV] n_neighbors=61 ..................................................
[CV] ......................................... n_neighbors=61 -   0.0s
[CV] n_neighbors=61 ..................................................
[CV] ......................................... n_neighbors=61 -   0.0s
[CV] n_neighbors=61 ..................................................
[CV] ......................................... n_neighbors=61 -   0.0s
[CV] n_neighbors=61 ..................................................
[CV] ......................................... n_neighbors=61 -   0.0s
[CV] n_neighbors=62 ..................................................
[CV] ......................................... n_neighbors=62 -   0.0s
[CV] n_neighbors=62 ..................................................
[CV] ......................................... n_neighbors=62 -   0.0s
[CV] n_neighbors=62 ..................................................
[CV] .

[CV] ......................................... n_neighbors=73 -   0.0s
[CV] n_neighbors=73 ..................................................
[CV] ......................................... n_neighbors=73 -   0.0s
[CV] n_neighbors=73 ..................................................
[CV] ......................................... n_neighbors=73 -   0.0s
[CV] n_neighbors=74 ..................................................
[CV] ......................................... n_neighbors=74 -   0.0s
[CV] n_neighbors=74 ..................................................
[CV] ......................................... n_neighbors=74 -   0.0s
[CV] n_neighbors=74 ..................................................
[CV] ......................................... n_neighbors=74 -   0.0s
[CV] n_neighbors=74 ..................................................
[CV] ......................................... n_neighbors=74 -   0.0s
[CV] n_neighbors=74 ..................................................
[CV] .

[CV] ......................................... n_neighbors=87 -   0.0s
[CV] n_neighbors=87 ..................................................
[CV] ......................................... n_neighbors=87 -   0.0s
[CV] n_neighbors=87 ..................................................
[CV] ......................................... n_neighbors=87 -   0.0s
[CV] n_neighbors=87 ..................................................
[CV] ......................................... n_neighbors=87 -   0.0s
[CV] n_neighbors=87 ..................................................
[CV] ......................................... n_neighbors=87 -   0.0s
[CV] n_neighbors=88 ..................................................
[CV] ......................................... n_neighbors=88 -   0.0s
[CV] n_neighbors=88 ..................................................
[CV] ......................................... n_neighbors=88 -   0.0s
[CV] n_neighbors=88 ..................................................
[CV] .

Best parameters set found on development set:

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

Grid scores on development set:

0.763 (+/-0.032) for {'n_neighbors': 1}
0.704 (+/-0.040) for {'n_neighbors': 2}
0.669 (+/-0.048) for {'n_neighbors': 3}
0.621 (+/-0.034) for {'n_neighbors': 4}
0.639 (+/-0.037) for {'n_neighbors': 5}
0.586 (+/-0.024) for {'n_neighbors': 6}
0.574 (+/-0.027) for {'n_neighbors': 7}
0.580 (+/-0.024) for {'n_neighbors': 8}
0.556 (+/-0.017) for {'n_neighbors': 9}
0.538 (+/-0.015) for {'n_neighbors': 10}
0.521 (+/-0.020) for {'n_neighbors': 11}
0.527 (+/-0.027) for {'n_neighbors': 12}
0.497 (+/-0.031) for {'n_neighbors': 13}
0.467 (+/-0.047) for {'n_neighbors': 14}
0.456 (+/-0.039) for {'n_neighbors': 15}
0.432 (+/-0.042) for {'n_neighbors': 16}
0.426 (+/-0.040) for {'n_neighbors': 17}
0.432 (+/-0.043) for {'n_neighbors': 18}
0.432 (+/-0.043) for {'n_n

[Parallel(n_jobs=1)]: Done 495 out of 495 | elapsed:    1.6s finished


In [14]:
# Affichage des courbes
#draw([0.6,0.9], clf_analysed, i_max, j_max, ecart_i, ecart_j)

In [15]:
# TODO copier coller le best classifier
#clf_analysed_demo = 

#clf_analysed_demo.fit(source_train_analysed, cible_train_analysed.values.ravel())



In [16]:

#for i in range(0,20):
#    print("prédit : ", clf_analysed_demo.predict(source_test_analysed.values[i]),"\n attendu : ",cible_test_analysed.values[i])



# Dataset ANALYSE 2

In [17]:

# Récupération des données analysé sans les données brutes (DONNEES ANALYSEES 2)
%store -r colonnes_source_analysed_2
%store -r colonnes_cible_analysed_2
%store -r source_analysed_2
%store -r cible_analysed_2


In [18]:
# cration d'un jeu de données de test et d'entrainement
source_train_analysed_2, source_test_analysed_2, cible_train_analysed_2, cible_test_analysed_2 = train_test_split(source_analysed_2, cible_analysed_2, test_size=0.7)


In [19]:
#[print('classe ',i,' : ',sum(cible_train==i)) for i in cible_train.values]

display(pd.Series.value_counts(activity_dataset_analysed['Action']))
display(source_train_analysed_2, source_test_analysed_2, cible_train_analysed_2, cible_test_analysed_2)


S'assoir (depuis position allongée)    66
Courrir                                60
S'allonger (depuis position assise)    59
Ne rien faire                          57
Marcher                                57
S'allonger (depuis position debout)    56
Sauter                                 56
S'assoir (chaise)                      52
Se lever (chaise)                      51
Se lever (depuis position allongée)    51
Name: Action, dtype: int64

,Lieux_Ecole,Lieux_Exterieur,Lieux_Maison,moyenne_x,moyenne_y,moyenne_z,moyenne_xyz,max_x,max_y,max_z,max_xyz,min_x,min_y,min_z,min_xyz,ecart_type_x,ecart_type_y,ecart_type_z,ecart_type_xyz
92,0,0,1,0.126182,0.192036,9.871858,3.396692,0.145569,0.218857,9.895798,9.895798,0.104874,0.172165,9.847916,0.104874,0.010719,0.010774,0.011567,4.617365
302,0,0,1,1.360152,7.945539,5.020307,4.775333,5.476257,11.770874,9.096130,11.770874,-4.914566,3.243927,0.420746,-4.914566,2.919134,2.111702,2.368193,3.656116
243,0,0,1,0.384815,8.053936,5.868273,4.769008,2.333878,10.728195,11.645966,11.645966,-0.844421,5.411880,2.899948,-0.844421,0.873992,1.424040,1.879163,3.552375
148,0,0,1,-17.692844,-13.648305,3.572773,-9.256125,4.062485,19.074387,28.052231,28.052231,-39.862732,-38.746780,-11.416183,-39.862732,15.567571,16.371873,8.347247,16.530515
480,0,0,1,-5.312485,5.422769,1.530579,0.546955,1.679062,8.805649,5.149292,8.805649,-11.475876,1.450668,-3.180130,-11.475876,4.767171,2.948567,2.945697,5.739142
132,0,0,1,-10.852849,-2.033254,1.832666,-3.684479,9.261505,24.973694,14.032974,24.973694,-39.862732,-19.186157,-12.349930,-39.862732,16.648060,10.866130,6.182278,12.971019
496,0,0,1,6.594297,2.815303,2.452825,3.954142,13.863159,9.417374,6.965302,13.863159,-2.958511,-2.942688,-2.976624,-2.976624,5.675736,4.118893,2.458236,4.620602
481,0,0,1,5.590468,5.000432,2.062810,4.217903,11.006866,9.125275,5.995651,11.006866,-0.533188,0.240402,-3.426743,-3.426743,4.471700,3.683558,2.802740,3.969791
338,0,0,1,0.812777,4.852948,8.599997,4.755241,0.940445,4.959366,8.679550,8.679550,0.718979,4.746292,8.527512,0.718979,0.056576,0.053284,0.043223,3.207102
499,0,0,1,0.338607,7.589641,-3.449959,1.492763,1.900528,10.075775,3.930649,10.075775,-1.191589,2.779449,-11.115723,-11.115723,0.941126,2.495702,5.007010,5.630115


,Lieux_Ecole,Lieux_Exterieur,Lieux_Maison,moyenne_x,moyenne_y,moyenne_z,moyenne_xyz,max_x,max_y,max_z,max_xyz,min_x,min_y,min_z,min_xyz,ecart_type_x,ecart_type_y,ecart_type_z,ecart_type_xyz
184,0,0,1,-0.330752,9.060756,3.672907,4.134304,1.749695,10.489975,9.008743,10.489975,-2.531143,7.236252,1.253922,-2.531143,0.829454,0.856550,1.982021,4.096386
555,0,0,1,-5.289323,5.138338,0.455462,0.101492,2.044174,10.066208,1.920715,10.066208,-12.528137,-1.574402,-1.641861,-12.528137,5.121620,4.124496,1.178598,5.732925
193,0,0,1,0.197769,9.163949,3.416669,4.259462,1.012283,9.941711,6.490051,9.941711,-0.583466,7.607361,1.921906,-0.583466,0.407192,0.665865,1.573595,3.870477
461,0,0,1,-0.060507,3.920886,9.096256,4.318878,0.338303,4.126190,9.695877,9.695877,-1.591415,3.716782,8.711868,-1.591415,0.391743,0.106445,0.255643,3.790237
172,0,0,1,0.140968,8.603944,4.700262,4.481725,2.564911,9.847137,7.899033,9.847137,-0.757034,6.813675,2.550385,-0.757034,0.832649,0.749898,1.013254,3.591461
161,1,0,0,-0.882197,3.327424,9.274263,3.906497,2.296768,6.843613,19.339722,19.339722,-4.029907,1.478195,4.960159,-4.029907,1.310193,1.426100,3.787183,4.845697
548,0,0,1,-5.356063,3.970864,1.945374,0.186725,1.875381,9.449692,4.621368,9.449692,-14.852905,-2.653000,-2.303848,-14.852905,5.431000,4.495392,1.742188,5.770692
400,0,0,1,-5.810960,3.352683,2.355920,-0.034119,1.012283,7.133301,9.209854,9.209854,-14.948669,-1.785095,-4.003738,-14.948669,4.956859,3.212379,4.570618,5.924553
203,0,0,1,-0.138197,8.885323,4.188261,4.311796,1.465973,10.647995,6.928192,10.647995,-1.696762,7.095001,2.294205,-1.696762,0.755834,0.925412,1.245430,3.842875
70,0,0,1,0.014909,5.334661,8.230452,4.526674,1.339081,6.732285,10.891785,10.891785,-1.415451,3.859238,6.727066,-1.415451,0.678686,0.757293,1.197400,3.544933


,Action
92,Ne rien faire
302,Courrir
243,Se lever (chaise)
148,Sauter
480,S'allonger (depuis position assise)
132,Sauter
496,S'assoir (depuis position allongée)
481,S'allonger (depuis position assise)
338,Ne rien faire
499,S'assoir (depuis position allongée)


,Action
184,S'assoir (chaise)
555,S'assoir (depuis position allongée)
193,S'assoir (chaise)
461,Ne rien faire
172,S'assoir (chaise)
161,Marcher
548,S'allonger (depuis position assise)
400,S'allonger (depuis position assise)
203,S'assoir (chaise)
70,Marcher


In [20]:
# Configuration des paramètres que l'on souhaite tester
tuned_parameters = [{
   'n_neighbors': np.array(range (1,100))
}]
scores = ['accuracy']

warnings.filterwarnings('ignore')


for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf_analysed_2 = GridSearchCV(KNeighborsClassifier(), tuned_parameters, cv=5, scoring=score,verbose=2)
    clf_analysed_2.fit(source_train_analysed_2, cible_train_analysed_2.values.ravel())
    
    
    print("Best parameters set found on development set:")
    print()
    print(clf_analysed_2.best_estimator_)
    
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf_analysed_2.grid_scores_:
          
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() / 2, params))
    print()
    
    print("Detailed classification report:")
    print()
    cible_true_analysed_2, cible_pred_analysed_2 = cible_test_analysed_2, clf_analysed_2.predict(source_test_analysed_2)
    print(classification_report(cible_true_analysed_2, cible_pred_analysed_2))
    
    
    print()
    
    

# Tuning hyper-parameters for accuracy

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......................................... n_neighbors=1 -   0.0s
[CV] n_neighbors=2 ...................................................
[CV] .......................................... n_neighbors=2 -   0.0s
[CV] n_neighbors=2 ...........................

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ......................................... n_neighbors=16 -   0.0s
[CV] n_neighbors=16 ..................................................
[CV] ......................................... n_neighbors=16 -   0.0s
[CV] n_neighbors=16 ..................................................
[CV] ......................................... n_neighbors=16 -   0.0s
[CV] n_neighbors=16 ..................................................
[CV] ......................................... n_neighbors=16 -   0.0s
[CV] n_neighbors=17 ..................................................
[CV] ......................................... n_neighbors=17 -   0.0s
[CV] n_neighbors=17 ..................................................
[CV] ......................................... n_neighbors=17 -   0.0s
[CV] n_neighbors=17 ..................................................
[CV] ......................................... n_neighbors=17 -   0.0s
[CV] n_neighbors=17 ..................................................
[CV] .

[CV] ......................................... n_neighbors=31 -   0.0s
[CV] n_neighbors=31 ..................................................
[CV] ......................................... n_neighbors=31 -   0.0s
[CV] n_neighbors=31 ..................................................
[CV] ......................................... n_neighbors=31 -   0.0s
[CV] n_neighbors=31 ..................................................
[CV] ......................................... n_neighbors=31 -   0.0s
[CV] n_neighbors=31 ..................................................
[CV] ......................................... n_neighbors=31 -   0.0s
[CV] n_neighbors=32 ..................................................
[CV] ......................................... n_neighbors=32 -   0.0s
[CV] n_neighbors=32 ..................................................
[CV] ......................................... n_neighbors=32 -   0.0s
[CV] n_neighbors=32 ..................................................
[CV] .

[CV] ......................................... n_neighbors=45 -   0.0s
[CV] n_neighbors=46 ..................................................
[CV] ......................................... n_neighbors=46 -   0.0s
[CV] n_neighbors=46 ..................................................
[CV] ......................................... n_neighbors=46 -   0.0s
[CV] n_neighbors=46 ..................................................
[CV] ......................................... n_neighbors=46 -   0.0s
[CV] n_neighbors=46 ..................................................
[CV] ......................................... n_neighbors=46 -   0.0s
[CV] n_neighbors=46 ..................................................
[CV] ......................................... n_neighbors=46 -   0.0s
[CV] n_neighbors=47 ..................................................
[CV] ......................................... n_neighbors=47 -   0.0s
[CV] n_neighbors=47 ..................................................
[CV] .

[CV] ......................................... n_neighbors=60 -   0.0s
[CV] n_neighbors=60 ..................................................
[CV] ......................................... n_neighbors=60 -   0.0s
[CV] n_neighbors=60 ..................................................
[CV] ......................................... n_neighbors=60 -   0.0s
[CV] n_neighbors=60 ..................................................
[CV] ......................................... n_neighbors=60 -   0.0s
[CV] n_neighbors=61 ..................................................
[CV] ......................................... n_neighbors=61 -   0.0s
[CV] n_neighbors=61 ..................................................
[CV] ......................................... n_neighbors=61 -   0.0s
[CV] n_neighbors=61 ..................................................
[CV] ......................................... n_neighbors=61 -   0.0s
[CV] n_neighbors=61 ..................................................
[CV] .

[CV] n_neighbors=75 ..................................................
[CV] ......................................... n_neighbors=75 -   0.0s
[CV] n_neighbors=75 ..................................................
[CV] ......................................... n_neighbors=75 -   0.0s
[CV] n_neighbors=75 ..................................................
[CV] ......................................... n_neighbors=75 -   0.0s
[CV] n_neighbors=76 ..................................................
[CV] ......................................... n_neighbors=76 -   0.0s
[CV] n_neighbors=76 ..................................................
[CV] ......................................... n_neighbors=76 -   0.0s
[CV] n_neighbors=76 ..................................................
[CV] ......................................... n_neighbors=76 -   0.0s
[CV] n_neighbors=76 ..................................................
[CV] ......................................... n_neighbors=76 -   0.0s
[CV] n

[CV] ......................................... n_neighbors=90 -   0.0s
[CV] n_neighbors=90 ..................................................
[CV] ......................................... n_neighbors=90 -   0.0s
[CV] n_neighbors=90 ..................................................
[CV] ......................................... n_neighbors=90 -   0.0s
[CV] n_neighbors=90 ..................................................
[CV] ......................................... n_neighbors=90 -   0.0s
[CV] n_neighbors=90 ..................................................
[CV] ......................................... n_neighbors=90 -   0.0s
[CV] n_neighbors=91 ..................................................
[CV] ......................................... n_neighbors=91 -   0.0s
[CV] n_neighbors=91 ..................................................
[CV] ......................................... n_neighbors=91 -   0.0s
[CV] n_neighbors=91 ..................................................
[CV] .

[Parallel(n_jobs=1)]: Done 495 out of 495 | elapsed:    1.4s finished


In [21]:
%matplotlib inline

# Affichage des courbes
#draw([0.50,0.80], clf_analysed_2, i_max, j_max, ecart_i, ecart_j)


In [22]:
# TODO copier coller le best classifier
#clf_analysed_2_demo = 

#clf_analysed_2_demo.fit(source_train_analysed_2, cible_train_analysed_2.values.ravel())



In [23]:

#for i in range(0,20):
#    print("prédit : ", clf_analysed_2_demo.predict(source_test_analysed_2.values[i]),"\n attendu : ",cible_test_analysed_2.values[i])
